In [69]:
### Settings

import json

LEAGUE = "vct-international"
YEAR = 2024

In [51]:
### UTIL FUNCTIONS CELL ###
# Get match time from events
def getMatchTime(events):
  i = 0
  while i < len(events):
    
    event = events[i]
    if "gamePhase" not in event.keys():
      i += 1
      continue

    if "phase" not in event["gamePhase"].keys():
      i += 1
      continue

    if event["gamePhase"]["phase"] != "GAME_STARTED":
      i += 1
      continue

    if "metadata" not in event.keys():
      i += 1
      continue

    if "wallTime" not in event["metadata"].keys():
      i += 1
      continue

    if not event["metadata"]["wallTime"] or event["metadata"]["wallTime"] == "":
      i += 1
      continue

    return event["metadata"]["wallTime"]
  return ""
  

def getMatchMap(events, maps):
  i = 0

  while i < len(events):
    event = events[i]

    if "configuration" not in event.keys():
      i += 1
      continue

    if "selectedMap" not in event["configuration"].keys():
      i += 1
      continue

    if "id" in event["configuration"]["selectedMap"].keys():
      id = event["configuration"]["selectedMap"]["id"]

      for map in maps:
        if map["id"] == id:
          return map["name"]
        
    if "fallback" not in event["configuration"]["selectedMap"].keys():
      i += 1
      continue

    if "guid" not in event["configuration"]["selectedMap"]["fallback"].keys():
      i += 1
      continue

    guid = event["configuration"]["selectedMap"]["fallback"]["guid"]
    for map in maps:
      if map["id"] == guid:
        return map["name"]
      if map["name"] == guid:
        return map["name"]
      if map["assetName"] == guid:
        return map["name"]
      if map["assetPath"] == guid:
        return map["name"]
      
    i += 1
  return ""


def getMatchPlayersAgents(events, agents):
  i = 0
  portfolios = {}
  while i < len(events):
    event = events[i]
    
    if "configuration" not in event.keys():
      i += 1
      continue

    if "players" not in event['configuration'].keys():
      i += 1
      continue

    players = event['configuration']['players']
    
    for player in players:
      if "playerId" not in player.keys():
        continue
      if "value" not in player["playerId"].keys():
        continue
      if "selectedAgent" not in player.keys():
        continue
      if "fallback" not in player["selectedAgent"].keys():
        continue
      if "guid" not in player["selectedAgent"]["fallback"].keys():
        continue
      portfolios[player["playerId"]["value"]] = {}
      agent_id = player["selectedAgent"]["fallback"]["guid"]
      for agent in agents:
        if agent["id"] == agent_id:
          portfolios[player["playerId"]["value"]]["agent_name"] = agent["name"]
          break

    if portfolios != {}:
      return portfolios
    i += 1

  return {}

def getMatchPlayersKDA(events):
  i = 0
  events_reversed = events[::-1]
  portfolios = {}
  while i < len(events_reversed):
    event = events_reversed[i]
    
    if "snapshot" not in event.keys():
      i += 1
      continue

    if "players" not in event['snapshot'].keys():
      i += 1
      continue

    if "metadata" not in event.keys():
      i += 1
      continue

    if "currentGamePhase" not in event["metadata"].keys():
      i += 1
      continue

    if "phase" not in event["metadata"]["currentGamePhase"].keys():
      i += 1
      continue

    if event["metadata"]["currentGamePhase"]["phase"] != "GAME_ENDED":
      i += 1
      continue

    players = event['snapshot']['players']

    for player in players:
      if "playerId" not in player.keys():
        continue
      if "value" not in player["playerId"].keys():
        continue
      playerId = player["playerId"]["value"]
      if "deaths" not in player.keys():
        continue
      if "assists" not in player.keys():
        continue
      if "kills" not in player.keys():
        continue
      if "scores" not in player.keys():
        continue
      if "combatScore" not in player["scores"].keys():
        continue
      if "totalScore" not in player["scores"]["combatScore"].keys():
        continue
      portfolios[playerId] = {
        "kills": player["kills"],
        "deaths": player["deaths"],
        "assists": player["assists"],
        "combatScore": player["scores"]["combatScore"]["totalScore"]
      }

    if portfolios != {}:
      return portfolios
    
    i += 1

  return {}


def getMatchRoundNumbers(events):
  i = 0
  events_reversed = events[::-1]

  while i < len(events_reversed):
    event = events_reversed[i]
    
    if "metadata" not in event.keys():
      i += 1
      continue

    if "currentGamePhase" not in event["metadata"].keys():
      i += 1
      continue

    if "phase" not in event["metadata"]["currentGamePhase"].keys():
      i += 1
      continue

    if "roundNumber" not in event["metadata"]["currentGamePhase"].keys():
      i += 1
      continue

    if event["metadata"]["currentGamePhase"]["phase"] != "GAME_ENDED":
      i += 1
      continue

    return event["metadata"]["currentGamePhase"]["roundNumber"]

  return ""

def getPlayerGunStats(events, weapons):
  i = 0
  portfolios = {"1": {}, "2": {}, "3": {}, "4": {}, "5": {}, "6": {}, "7": {}, "8": {}, "9": {}, "10": {}}

  while i < len(events):
    event = events[i]

    if "playerDied" not in event.keys():
      i += 1
      continue

    if "killerId" not in event["playerDied"].keys() or "weapon" not in event["playerDied"].keys():
      i += 1
      continue

    if "value" not in event["playerDied"]["killerId"].keys():
      i += 1
      continue

    weapon_id = event["playerDied"]["weapon"]["fallback"]["guid"]
    weapon_name = ""
    for weapon in weapons:
      if weapon["id"] == weapon_id:
        weapon_name = weapon["name"]
        break

    killerId = str(event["playerDied"]["killerId"]["value"])
    if killerId not in portfolios.keys() or weapon_name not in portfolios[killerId].keys():
      portfolios[killerId][weapon_name] = 1
    else:
      portfolios[killerId][weapon_name] += 1

    i += 1

  return portfolios

def getPlayersGunDamage(events, weapons):
  i = 0
  portfolios = {"1": {}, "2": {}, "3": {}, "4": {}, "5": {}, "6": {}, "7": {}, "8": {}, "9": {}, "10": {}}

  while i < len(events):
    event = events[i]

    if "damageEvent" not in event.keys():
      i += 1
      continue

    if "causerId" not in event["damageEvent"].keys() or "victimId" not in event["damageEvent"].keys() or "damageDealt" not in event["damageEvent"].keys() or "weapon" not in event["damageEvent"].keys():
      i += 1
      continue

    if "value" not in event["damageEvent"]["causerId"].keys() or "value" not in event["damageEvent"]["victimId"].keys() or "fallback" not in event["damageEvent"]["weapon"].keys():
      i += 1
      continue

    if "guid" not in event["damageEvent"]["weapon"]["fallback"].keys():
      i += 1
      continue

    damage = event["damageEvent"]["damageDealt"]
    dealerId = str(event["damageEvent"]["causerId"]["value"])
    weaponId = event["damageEvent"]["weapon"]["fallback"]["guid"]
    weapon_name = ""
    for weapon in weapons:
      if weapon["id"] == weaponId:
        weapon_name = weapon["name"]
        break

    if dealerId not in portfolios.keys() or weapon_name not in portfolios[dealerId].keys():
      portfolios[dealerId][weapon_name] = int(damage)
    else:
      portfolios[dealerId][weapon_name] += int(damage)

    i += 1

  return portfolios


In [52]:
#### Get all teams, make sure no duplicates ####
import os

teams_file = os.path.join(LEAGUE, "esports-data", "teams.json")

teams = json.load(open(teams_file))
teams = teams[::-1]

existing_team_ids = []
all_teams = []
for team in teams:
  if team['id'] not in existing_team_ids:
    if team['home_league_id'] != "107254585505459304" and team['home_league_id'] != "109551178413356399":
      existing_team_ids.append(team['id'])
      all_teams.append(team)

json.dump(all_teams, open(os.path.join(LEAGUE, 'teams.json'), 'w'), indent=4)

In [53]:
#### Get all players, make sure no duplicates
import os

players_file = os.path.join(LEAGUE, "esports-data", "players.json")
players = json.load(open(players_file))
players = players[::-1]

existing_player_ids = []
all_players = []
for player in players:
  if player['id'] not in existing_player_ids:
    existing_player_ids.append(player['id'])
    all_players.append(player)

json.dump(all_players, open(os.path.join(LEAGUE, 'players.json'), 'w'), indent=4)

In [55]:
### BASIC PORTFOLIOS (WITHOUT TEAM INFO) ###

players_file = os.path.join(LEAGUE, "players.json")

players = json.load(open(players_file))



for player in players:
  playerId = player['id']
  portfolio_file_name = playerId + ".json"
  portfolio_path = os.path.join(LEAGUE, "portfolios", portfolio_file_name)
  porfolio_dict = {
    "id": playerId,
    "first_name": player['first_name'],
    "last_name": player['last_name'],
    "status": player['status'],
    "current_team_id": player['home_team_id'],
    "photo_url": player['photo_url'],
    "in_game_name": player['handle'],
  }
  json.dump(porfolio_dict, open(portfolio_path, 'w'), indent=4)


In [56]:
#### Update portfolios to include team and league information ####
portfolios_files = os.listdir(os.path.join(LEAGUE, "portfolios"))

teams_file = os.path.join(LEAGUE, "teams.json")
teams = json.load(open(teams_file))

leauges = json.load(open(os.path.join(LEAGUE, "esports-data", "leagues.json")))

for portfolio_file in portfolios_files:
  data = json.load(open(os.path.join(LEAGUE, "portfolios", portfolio_file)))
  teams_id = data['current_team_id']
  
  for team in teams:
    if team['id'] == teams_id:
      data['current_team_acronym'] = team['acronym']
      data['current_team_name'] = team['name']
      home_league_id = team['home_league_id']
      
      for league in leauges:
        if league['league_id'] == home_league_id:
          data['current_league_id'] = league['league_id']
          data['current_league_name'] = league['name']
          data['region'] = league['region']
          break
      
      break

  json.dump(data, open(os.path.join(LEAGUE, "portfolios", portfolio_file), 'w'), indent=4)

In [ ]:
import os
event_file = "data.json"
mapping_file = os.path.join(LEAGUE, "esports-data", "mapping_data.json")

matches = json.load(open(mapping_file))
maps = json.load(open("static/maps.json"))
agents = json.load(open("static/agents.json"))
weapons = json.load(open("static/weapons.json"))

for match in matches:

  if 'platformGameId' in match.keys():
    platform_game_id = match['platformGameId']
    print("Platform Game ID: ", platform_game_id)

    file_path = os.path.join(LEAGUE, "games", str(YEAR), platform_game_id + ".json")

    if not os.path.exists(file_path):
      print("Game", platform_game_id, "does not exist.")
      continue

    events = json.load(open(file_path))
    game_time = getMatchTime(events)
    map = getMatchMap(events, maps)

    players_agents = getMatchPlayersAgents(events, agents)
    kda = getMatchPlayersKDA(events)
    round_number = getMatchRoundNumbers(events)
    players_gun_stats = getPlayerGunStats(events, weapons)
    players_gun_damage = getPlayersGunDamage(events, weapons)
    
    print("Game Time: ", game_time)
    print("Map: ", map)
    print("Players Agents: ", players_agents)
    print("Players KDA", kda)
    print("Round Number", round_number)
    print("Players Gun Stats", players_gun_stats)

    if game_time == "" or map == "" or players_agents == {} or kda == {} or round_number == "" or players_gun_stats == {} or players_gun_damage == {}:
      print("Game", platform_game_id, "is missing some data.")
      continue

    ### Prolly do this after all to update the portfolios
    participants = match["participantMapping"]

    for index, player_id in participants.items():
      portfolio_file = player_id + ".json"
      if not os.path.exists(os.path.join(LEAGUE, "portfolios", portfolio_file)):
        print("Portfolio for player", player_id, "does not exist.")
        continue

      portfolio = json.load(open(os.path.join(LEAGUE, "portfolios", portfolio_file)))

      if "game_records" not in portfolio.keys():
        portfolio["game_records"] = []

      for record in portfolio["game_records"]:
        if record["platform_game_id"] == platform_game_id:
          print("Game already exists in portfolio.")
          break

      portfolio["game_records"].append({
        "platform_game_id": platform_game_id,
        "game_time": game_time,
        "map": map,
        "agents": players_agents[int(index)],
        "kda": kda[int(index)],
        "round_count": round_number,
        "gun_kills": players_gun_stats[index],
        "gun_damage": players_gun_damage[index]
      })
      print(portfolio["id"])
      print(portfolio["game_records"])

      json.dump(portfolio, open(os.path.join(LEAGUE, "portfolios", portfolio_file), 'w'), indent=4)


In [104]:
######## SORT AND CREATE A METADATA PROFILE FILE ##############

import os

LEAGUE = "vct-international"


portfolios_dir = os.path.join(LEAGUE, "portfolios")
portfolios_files = os.listdir(portfolios_dir)
portfolios_files = [file for file in portfolios_files if file.endswith(".json")]

summary_file = os.path.join(LEAGUE, "profiles.json")
non_active_players_file = os.path.join(LEAGUE, "non_active_players.json")

profiles = {
  "league": LEAGUE,
  "players_sort_by_better_acs_first": []
}

non_active_players = {
  "league": LEAGUE,
  "players": []
}
for portfolio_file in portfolios_files:
  portfolio = json.load(open(os.path.join(portfolios_dir, portfolio_file)))

  temp = {}

  temp['id'] = portfolio['id']
  temp['first_name'] = portfolio['first_name']
  temp['last_name'] = portfolio['last_name']
  temp['status'] = portfolio['status']
  if "current_team_acronym" in portfolio.keys():
    temp['current_team_acronym'] = portfolio['current_team_acronym']
  else:
    temp['current_team_acronym'] = ""
  if "current_team_name" in portfolio.keys():
    temp['current_team_name'] = portfolio['current_team_name']
  else:
    temp['current_team_name'] = ""
  if "current_league_name" in portfolio.keys():
    temp['current_league_name'] = portfolio['current_league_name']
  else: 
    temp['current_league_name'] = ""

  if "region" in portfolio.keys():
    temp['region'] = portfolio['region']
  else:
    temp['region'] = ""

  acs = 0
  game_count = 0
  kills = 0
  deaths = 0
  assists = 0
  if "game_records" not in portfolio.keys():
    non_active_players["players"].append(temp)
    continue
  for record in portfolio['game_records']:
    if 'kda' not in record.keys() or 'round_count' not in record.keys() or 'combatScore' not in record['kda'].keys():
      continue

    if record['round_count'] == 0:
      continue
    acs += record['kda']['combatScore']/record['round_count']
    game_count += 1
    kills += record['kda']['kills']/record['round_count']
    deaths += record['kda']['deaths']/record['round_count']
    assists += record['kda']['assists']/record['round_count']

  if game_count == 0:
    continue

  stat_temp  = {}
  stat_temp['acs'] = round(acs/game_count, 2)
  stat_temp['kills'] = round(kills/game_count, 2)
  stat_temp['deaths'] = round(deaths/game_count, 2)  
  stat_temp['assists'] = round(assists/game_count, 2)
  temp['general_statistics'] = stat_temp

  agent_temp = {}
  for record in portfolio["game_records"]:
    if 'kda' not in record.keys() or 'round_count' not in record.keys() or 'combatScore' not in record['kda'].keys():
      continue
    if "agents" not in record.keys() or "agent_name" not in record["agents"].keys():
      continue
    if record['round_count'] == 0:
      continue
    agent = record["agents"]["agent_name"]
    if agent not in agent_temp.keys():
      agent_temp[agent] = {
        "acs": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "game_count": 0
      }
    agent_temp[agent]["acs"] += record['kda']['combatScore']/record['round_count']
    agent_temp[agent]["kills"] += record['kda']['kills']/record['round_count']
    agent_temp[agent]["deaths"] += record['kda']['deaths']/record['round_count']
    agent_temp[agent]["assists"] += record['kda']['assists']/record['round_count']
    agent_temp[agent]["game_count"] += 1


  agent_stats = []
  for agent in agent_temp.keys():
    agent_stats.append({
      "agent": agent,
      "acs": round(agent_temp[agent]["acs"]/agent_temp[agent]["game_count"], 2),
      "kills": round(agent_temp[agent]["kills"]/agent_temp[agent]["game_count"], 2),
      "deaths": round(agent_temp[agent]["deaths"]/agent_temp[agent]["game_count"], 2),
      "assists": round(agent_temp[agent]["assists"]/agent_temp[agent]["game_count"], 2),
      "rank": 0
    })

  agent_stats = sorted(agent_stats, key=lambda x: x["acs"], reverse=True)
  for index, agent in enumerate(agent_stats):
    agent["rank"] = index + 1

  temp['agent_statistics'] = agent_stats

  map_temp = {}
  for record in portfolio['game_records']:
    if 'map' not in record.keys() or 'round_count' not in record.keys() or 'combatScore' not in record['kda'].keys():
      continue
    map = record['map']
    if record['round_count'] == 0:
      continue
    if map not in map_temp.keys():
      map_temp[map] = {
        "acs": 0,
        "kills": 0,
        "deaths": 0,
        "assists": 0,
        "game_count": 0
      }
    
    map_temp[map]["acs"] += record['kda']['combatScore']/record['round_count']
    map_temp[map]["kills"] += record['kda']['kills']/record['round_count']
    map_temp[map]["deaths"] += record['kda']['deaths']/record['round_count']
    map_temp[map]["assists"] += record['kda']['assists']/record['round_count']
    map_temp[map]["game_count"] += 1

  map_stats = []
  for map in map_temp.keys():
    if map_temp[map]["game_count"] == 0:
      continue
    map_stats.append({
      "map": map,
      "acs": round(map_temp[map]["acs"]/map_temp[map]["game_count"], 2),
      "kills": round(map_temp[map]["kills"]/map_temp[map]["game_count"], 2),
      "deaths": round(map_temp[map]["deaths"]/map_temp[map]["game_count"], 2),
      "assists": round(map_temp[map]["assists"]/map_temp[map]["game_count"], 2),
      "rank": 0
    })

  map_stats = sorted(map_stats, key=lambda x: x["acs"], reverse=True)
  for index, map in enumerate(map_stats):
    map["rank"] = index + 1

  temp['map_statistics'] = map_stats

  profiles["players_sort_by_better_acs_first"].append(temp)


profiles["players_sort_by_better_acs_first"] = sorted(profiles["players_sort_by_better_acs_first"], key=lambda x: x["general_statistics"]["acs"], reverse=True)

json.dump(profiles, open(summary_file, 'w'), indent=4)
json.dump(non_active_players, open(non_active_players_file, 'w'), indent=4)


In [ ]:
### SORT Game records ###
import os
from dateutil import parser

LEAGUE = "vct-challengers"

portfolios_dir = os.path.join(LEAGUE, "portfolios")
portfolios_files = os.listdir(portfolios_dir)
portfolios_files = [file for file in portfolios_files if file.endswith(".json")]

for portfolio_file in portfolios_files:
  portfolio = json.load(open(os.path.join(portfolios_dir, portfolio_file)))
  if "game_records" not in portfolio.keys():
    continue

  portfolio["game_records"] = sorted(portfolio["game_records"], key=lambda x: parser.isoparse(x["game_time"]))

  print(portfolio["game_records"])
  json.dump(portfolio, open(os.path.join(portfolios_dir, portfolio_file), 'w'), indent=4)

In [81]:
#### Update portfolios folder so to only include players with game records and is still staying in the LEAGUE (in case of switch LEAGUE) ####

import os

game_changers_dir = os.path.join("game-changers", "portfolios")
vct_challengers_dir = os.path.join("vct-challengers", "portfolios")
vct_international_dir = os.path.join("vct-international", "portfolios")
game_changers_files = [os.path.join(game_changers_dir, file) for file in os.listdir(game_changers_dir)]
vct_challengers_files = [os.path.join(vct_challengers_dir, file) for file in os.listdir(vct_challengers_dir)]
vct_international_files = [os.path.join(vct_international_dir, file) for file in os.listdir(vct_international_dir)]



for file in game_changers_files:
  portfolio = json.load(open(file))
  if "game_records" not in portfolio.keys():
    print("No game records, removing ", file)
    os.remove(file)
    
  else:
    for t_file in vct_international_files:
      if t_file.split("/")[-1] == file.split("/")[-1]:
        current_file = json.load(open(file))
        compare_file = json.load(open(t_file))
        if "game_records" not in compare_file.keys():
          continue
        current_latest_game = current_file["game_records"][-1]["game_time"]
        compare_latest_game = compare_file["game_records"][-1]["game_time"]

        current_latest_date = parser.isoparse(current_latest_game) 
        compare_latest_date = parser.isoparse(compare_latest_game)

        if current_latest_date < compare_latest_date:
          print("Not the latest portfolio, removing ", file)
          #os.remove(file)
          
        
        
        

In [ ]:
import os

game_changers_dir = os.path.join("game-changers", "portfolios")
vct_challengers_dir = os.path.join("vct-challengers", "portfolios")
vct_international_dir = os.path.join("vct-international", "portfolios")
game_changers_files = [os.path.join(game_changers_dir, file) for file in os.listdir(game_changers_dir)]
game_changers_files = [file for file in game_changers_files if file.endswith(".json")]
vct_challengers_files = [os.path.join(vct_challengers_dir, file) for file in os.listdir(vct_challengers_dir)]
vct_challengers_files = [file for file in vct_challengers_files if file.endswith(".json")]
vct_international_files = [os.path.join(vct_international_dir, file) for file in os.listdir(vct_international_dir)]
vct_international_files = [file for file in vct_international_files if file.endswith(".json")]

all_files = game_changers_files + vct_challengers_files + vct_international_files

metadata = []

for file in all_files:
  print(file)
  portfolio = json.load(open(file))

  result = {}
  result["id"] = portfolio["id"]
  result["first_name"] = portfolio["first_name"]
  result["last_name"] = portfolio["last_name"]

  metadata.append(result)

json.dump(metadata, open("metadata.json", 'w'), indent=4)

In [ ]:
import os 

LEAGUE = "vct-challengers"

portfolios_dir = os.path.join(LEAGUE, "portfolios")
profiles_file = os.path.join(LEAGUE, "profiles.json")

portfolios_files = [os.path.join(portfolios_dir, file) for file in os.listdir(portfolios_dir)]


profiles = json.load(open(profiles_file))
esport_mapping_file = os.path.join(LEAGUE, "esports-data", "mapping_data.json")
esport_mapping = json.load(open(esport_mapping_file))

teams_file = os.path.join(LEAGUE, "teams.json")
teams = json.load(open(teams_file))

result_profiles = []

for profile in profiles["players_sort_by_better_acs_first"]:
  temp = profile
  portfolio_file = profile["id"] + ".json"
  portfolio_dir = os.path.join(portfolios_dir, portfolio_file)

  if (not os.path.exists(portfolio_dir)):
    continue
  portfolio = json.load(open(portfolio_dir))
  lastest_game = portfolio["game_records"][-1]
  lastest_game_id = lastest_game["platform_game_id"]

  match = None
  for e in esport_mapping:
    if e["platformGameId"] == lastest_game_id:
      match = e
      break

  if match:
    match_keys = list(match["teamMapping"].keys())
    team1 = match["teamMapping"][match_keys[0]]
    team2 = match["teamMapping"][match_keys[1]]
    team1_name = ""
    team2_name = ""

    for team in teams:
      if team["id"] == team1:
        team1_name = team["name"]
      if team["id"] == team2:
        team2_name = team["name"]

    if team1_name != "" and team2_name != "":
      print("Yes access")
      temp["latest_game"] = {
        "team1": team1_name,
        "team2": team2_name,
        "map": lastest_game["map"],
        "player_agent": lastest_game["agents"]["agent_name"],
        "player_kda": lastest_game["kda"],
        "player_round_count": lastest_game["round_count"],
      }

  if len(portfolio["game_records"]) >= 2:
    second_lastest_game = portfolio["game_records"][-2] 
    second_lastest_game_id = second_lastest_game["platform_game_id"] 

    match = None

    for e in esport_mapping:
      if e["platformGameId"] == second_lastest_game_id:
        match = e
        break

    if match:
      match_keys = list(match["teamMapping"].keys())
      team1 = match["teamMapping"][match_keys[0]]
      team2 = match["teamMapping"][match_keys[1]]

      team1_name = ""
      team2_name = ""

      for team in teams:
        if team["id"] == team1:
          team1_name = team["name"]
        if team["id"] == team2:
          team2_name = team["name"]

      if team1_name and team2_name:
        temp["second_latest_game"] = {
          "team1": team1_name,
          "team2": team2_name,
          "map": second_lastest_game["map"],
          "player_agent": second_lastest_game["agents"]["agent_name"],
          "player_kda": second_lastest_game["kda"],
          "player_round_count": second_lastest_game["round_count"],
        }

  result_profiles.append(temp)

result = {
  "league": LEAGUE,
  "players_sort_by_better_acs_first": result_profiles
}

output_dir = os.path.join(LEAGUE, "modified_profiles.json")

json.dump(result, open(output_dir, 'w'), indent=4)
